### Cleaning Overview
- Input: Raw GA4 event-level Parquet snapshot
- Objective: Prepare analytics-ready dataset
- Key cleaning steps:
  - Null handling
  - Deduplication
  - Type standardization
  - Feature derivation
- Output: Clean Parquet for downstream analysis


In [3]:
import pandas as pd
from pathlib import Path

In [4]:
RAW_DATA_PATH = Path("../data/raw")
CLEAN_DATA_PATH = Path("../data/cleaned")

raw_file = RAW_DATA_PATH / "fact_events_raw.parquet"

df = pd.read_parquet(raw_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2251084 entries, 0 to 2251083
Data columns (total 14 columns):
 #   Column                Dtype              
---  ------                -----              
 0   user_id               object             
 1   event_name            object             
 2   transaction_id        object             
 3   event_date            datetime64[ns]     
 4   event_ts              datetime64[us, UTC]
 5   engagement_time_msec  Int64              
 6   session_id            Int64              
 7   device                object             
 8   country               object             
 9   region                object             
 10  city                  object             
 11  traffic_sname         object             
 12  traffic_smedium       object             
 13  traffic_source        object             
dtypes: Int64(2), datetime64[ns](1), datetime64[us, UTC](1), object(10)
memory usage: 244.7+ MB
